In [46]:
import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.linear_model import LogisticRegression
from scipy import stats

plt.style.use('seaborn-whitegrid')

__Задача 1__

Объясняемая переменная `y` зависит от двух категориальных факторов `A` и `B`, причём фактор `A` имеет 3 уровня, а фактор `B` - 4 уровня. Результаты наблюдений приведены в таблице:

```
y = [[2.68, 3.29, 2.88, 4.45],
     [4.12, 4.96, 5.09, 5.22],
     [5.52, 4.50, 5.42, 5.29]]
```

С помощью двухфакторного дисперсионного анализа проверьте влияние каждого из факторов на переменную `y`.

In [4]:
y = np.array([[2.68, 3.29, 2.88, 4.45],
              [4.12, 4.96, 5.09, 5.22],
              [5.52, 4.50, 5.42, 5.29]])

#### Проверим нулевую гипотезу о равенстве средних значений на каждом уровне:

In [38]:
y_k = y.mean(axis=0)
y_m = y.mean(axis=1)
y_mean = y.mean()

print('Среднее значение по столбцам:', y_k)
print('Среднее значение по строкам:', y_m)
print('Среднее по матрице:', y_mean)

Среднее значение по столбцам: [4.10666667 4.25       4.46333333 4.98666667]
Среднее значение по строкам: [3.325  4.8475 5.1825]
Среднее по матрице: 4.451666666666666


Нулевую гипотезу можно отвергнуть, т.к. средние значения не равны

In [18]:
k = len(y[0])
m = len(y)
k, m

(4, 3)

Посчитаем суммы квадратов отклонений:
$$S_A^2 = k \cdot \displaystyle\sum_{i=1}^m \left( \overline{y_{i \ast}} - \overline{y} \right)^2, \:
S_B^2 = m \cdot \displaystyle\sum_{j=1}^k \left( \overline{y_{\ast j}} - \overline{y} \right)^2, \:
S_w^2 = \displaystyle\sum_{i=1}^m \displaystyle\sum_{j=1}^k \left( y_{ij} - \overline{y_{i \ast}} - \overline{y_{\ast j}} + \overline{y} \right)^2.$$

In [39]:
s_a = 0
for i in range(m):
    s_a += (y_m[i] - y_mean)**2

s_a = s_a * k
s_a

7.8407166666666654

In [40]:
s_b = 0
for j in range(k):
    s_b += (y_k[j] - y_mean)**2

s_b = s_b * m
s_b

1.338166666666669

In [42]:
s_w = 0
for i in range(m):
    for j in range(k):
        s_w += (y[i, j] - y_m[i] - y_k[j] + y_mean)**2
s_w

1.9298833333333327

Посчитаем дисперсии:
$$\sigma_A^2 = \dfrac{S_A^2}{m - 1}, \: \sigma_B^2 = \dfrac{S_B^2}{k - 1}, \:
\sigma_w^2 = \dfrac{S_{res}^2}{(k - 1) (m - 1)}.$$

In [43]:
disp_a = s_a/(m-1)
disp_b = s_b/(k-1)
disp_w = s_w/((m-1)*(k-1))

disp_a, disp_b, disp_w

(3.9203583333333327, 0.4460555555555563, 0.32164722222222214)

Вычислим статистики:
$$F_A = \dfrac{\sigma_A^2}{\sigma_w^2}, \: F_B = \dfrac{\sigma_B^2}{\sigma_w^2}.$$

In [44]:
f_a = disp_a / disp_w
f_b = disp_b / disp_w
f_a, f_b

(12.188379262995173, 1.386785038819275)

Выберем уровень значимости и вычислим критическое значение

In [47]:
n = k * m
k1_a = m - 1
k2_a = n - m
k1_b = k - 1
k2_b = n - k

alpha = 0.05

f_a_crit = stats.f.ppf(1 - alpha, k1_a, k2_a)
f_b_crit = stats.f.ppf(1 - alpha, k1_b, k2_b)
f_a_crit, f_b_crit

(4.25649472909375, 4.06618055135116)

* $F_A > F_{crit, \: A}$ - следовательно __фактор А влияет__ на переменную y.
* $F_A < F_{crit, \: B}$ - следовательно __фактор B не влияет__ на переменную y.

__Задача 2__

С помощью критерия Стьюдента для двух независимых выборок проверьте гипотезу о равенстве среднего роста футболистов и хоккеистов, основываясь на результатах измерений:

```
football_players = [173, 175, 180, 178, 177, 185, 183, 182]
hockey_players = [177, 179, 180, 188, 177, 172, 171, 184, 180]
```

In [48]:
football_players = np.array([173, 175, 180, 178, 177, 185, 183, 182])
hockey_players = np.array([177, 179, 180, 188, 177, 172, 171, 184, 180])

Средние значения выборок:

In [49]:
f_mean = football_players.mean()
h_mean = hockey_players.mean()
f_mean, h_mean

(179.125, 178.66666666666666)

Дисперсия выборок:

In [59]:
f_std = football_players.std(ddof=1)
h_std = hockey_players.std(ddof=1)
f_std, h_std

(4.120939559996343, 5.338539126015656)

Для проверки гипотезы рассмотрим статистику
$$T = \dfrac{\overline{X_1} -\overline{X_2}}{\sqrt{\frac{\sigma_1^2}{n_1} + \frac{\sigma_2^2}{n_2}}}$$

In [60]:
t = (f_mean - h_mean) / (f_std**2 / len(football_players) + h_std**2 / len(hockey_players))**0.5
t

0.19928601397363732

Как я понимаю, если данная статистика равна или очень близка к 0, то гипотеза верная. В данном случае гипотеза не подтвердилась